In [ ]:
!pip install -U scikit-fuzzy
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 15.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Student_Marks.csv')

In [ ]:
df.describe()

,number_courses,time_study,Marks
count,100.000000,100.000000,100.000000
mean,5.290000,4.077140,24.417690
std,1.799523,2.372914,14.326199
min,3.000000,0.096000,5.609000
25%,4.000000,2.058500,12.633000
50%,5.000000,4.022000,20.059500
75%,7.000000,6.179250,36.676250
max,8.000000,7.957000,55.299000


In [ ]:
hours = ctrl.Antecedent(np.arange(df.time_study.min(), df.time_study.max()), 'Study')
cmd = ctrl.Consequent(np.arange(df.Marks.min(), df.Marks.max()+1), 'Marks')

In [ ]:
time_study_data = df.time_study
mean_time_study = time_study_data.mean()
std_time_study = time_study_data.std()

In [ ]:
hours['low'] = fuzz.gaussmf(hours.universe, mean_time_study - std_time_study, std_time_study / 2)  # Low: Mean - std, Sigma = std/2
hours['medium'] = fuzz.gaussmf(hours.universe, mean_time_study, std_time_study / 2)  # Medium: Mean, Sigma = std/2
hours['high'] = fuzz.gaussmf(hours.universe, mean_time_study + std_time_study, std_time_study / 2)  # High: Mean + std, Sigma = std/2

In [ ]:
marks_data = df.Marks
mean_marks = marks_data.mean()
std_marks = marks_data.std()
cmd = ctrl.Consequent(np.arange(marks_data.min(), marks_data.max() + 1), 'Marks')  # +1 to include max value
cmd['low'] = fuzz.gaussmf(cmd.universe, mean_marks - std_marks, std_marks / 2)
cmd['medium'] = fuzz.gaussmf(cmd.universe, mean_marks, std_marks / 2)
cmd['high'] = fuzz.gaussmf(cmd.universe, mean_marks + std_marks, std_marks / 2)


In [ ]:
rule1 = ctrl.Rule(hours['low'] | hours['medium'], cmd['low'])  # If hours are low or medium, then Marks are low
rule2 = ctrl.Rule(hours['medium'], cmd['medium'])  # If hours are medium, then Marks are medium
rule3 = ctrl.Rule(hours['medium'] | hours['high'], cmd['high'])

In [ ]:
cmd_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
cmd_output = ctrl.ControlSystemSimulation(cmd_ctrl)

In [ ]:
def generateOutput(study_hours):
    cmd_output.input['Study'] = study_hours  # Use 'Study' as input variable name

    cmd_output.compute()

    print(f"Predicted Marks: {cmd_output.output['Marks']:.2f}")  # Print predicted Marks
    return cmd_output.output['Marks']  # Return predicted Marks

In [ ]:
hrs = float(input("Input Hours: "))
generateOutput(hrs)

Input Hours: 1
Predicted Marks: 15.71


15.705707965390419